<a href="https://colab.research.google.com/github/Bearbrice/maDLC_MM_hands/blob/main/Berclaz_Colab_TrainNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands tracking with DeepLabCut - Training and evaluation of the network

> Using multi-animal DLC (maDLC)

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- cross evaluate inference parameters
- analyze novel videos
- assemble tracklets
- create quality check plots

## Credits and references:
* Cloned from https://github.com/AlexEMG/DeepLabCut

* Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

* Paper: https://www.nature.com/articles/s41596-019-0176-0

* Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf

* Adaptation: Brice Berclaz






## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

Set up DLC!

In [ ]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

#GUIs don't work on the cloud, so label your data locally on your computer! This will suppress the GUI support
import os
os.environ["DLClight"]="True"
import deeplabcut
deeplabcut.__version__

## Get the data from repository

In [ ]:
# Clone the github repository
!git clone -l -s git://github.com/Bearbrice/maDLC_MM_hands.git
#cloned-DLC_MM_hands
#%cd cloned-DLC_MM_hands
!ls

In [ ]:
#create a path variable that links to the config file:
path_config_file = '/content/maDLC_MM_hands/maDLC_MM_hands-Brice-2021-07-08/config.yaml'
print(path_config_file)


VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/maDLC_MM_hands/maDLC_MM_hands-Brice-2021-07-08/videos'] #Enter the list of videos or folder to analyze.
print(videofile_path)

# Loading example data set:
# deeplabcut.load_demo_data(path_config_file)

## Create training dataset

In [ ]:
# Note: if you are using the demo data (i.e. examples/Reaching-Mackenzie-2018-08-30/), first delete the folder called dlc-models! 
#Then, run this cell. There are many more functions you can set here, including which netowkr to use!
#check the docstring for full options you can do!
#deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

#if you labeled on Windows, please set the windows2linux=True:
deeplabcut.create_multianimaltraining_dataset(path_config_file, windows2linux=False)

## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network
shuffle = 1
deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=1000, saveiters=10000, maxiters=20000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: for maDLC, this is several steps. 
 - First, we evaluate the pose estimation performance, and then we can cross-valudate optimal inference parameters.

- This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)

In [ ]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True,c_engine=False)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

#%matplotlib notebook
#deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train! 

NOTE: this optimized part detection for video analysis. It cannot optimze for tracking, as this is use-case dependent. Please check the docs on how you can set the best parameters and modify/test before "final" tracking parameters. You can use COLAB to analyze videos, but afterwards we recommend using the outputs/proejct folder locally to run the final steps! They do not require a GPU. 

In [ ]:
#Cross-validate for Animal Assembly & Tracking:
deeplabcut.evaluate_multianimal_crossvalidate(
            path_config_file,
            Shuffles=[shuffle],
            edgewisecondition=True,
            leastbpts=1,
            init_points=20,
            n_iter=100,
            target='rpck_train',
        )

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
print("Start Analyzing my video(s)!")
scorername = deeplabcut.analyze_videos(path_config_file, 
                                       videofile_path, 
                                       shuffle=shuffle, 
                                       videotype=VideoType, 
                                       c_engine=False)

## The steps below work on a single video at a time.
- Here you can create a video to check the pose estimation detection quality! If this looks good, proceed to tracklet conversions with the interactive GUI (ouside of COLAB for now), or if you know your optimal parameters, you can automate this and run the additional steps shown in a few cells down.

In [ ]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
##  don't look good, don't proceed with tracking - add more data to your training set and re-train!

#let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = 'content/maDLC_MM_hands/...'

from deeplabcut.utils import auxiliaryfunctions
scorername, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(path_config_file, shuffle, trainFraction=0)
print("scorename is: "+scorername)

deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], scorername)

#Again, if this does not look perfect, do not proceed! Retrain with more diverse data.

## Convert Detections to Tracklets:

- The idea is that you test and adapt hyperparameters for tracking outside of COLAB. Once you have good parameters, this can be automated on future videos. Shown here!

- I.e., instead of always doing an interactive parameter setting step, you can simply convert tracklets to .h5 files using these parameters (see GitHub for more info).

In [ ]:
#assemble tracklets:
#read the docs: which tracker to test out (you can run this many times to try multiple):
tracktype= 'box' #box, skeleton, ellipse

deeplabcut.convert_detections2tracklets(path_config_file, Specific_videofile, videotype=VideoType,
                                                    shuffle=shuffle, track_method=tracktype, overwrite=True)

## Now you should manually verify the tracks and correct them if needed! [currently only working outside of COLAB]

In [ ]:
''' here is the code you would need:
os.environ["DLClight"]="False"
import deeplabcut

#ATTENTION:
picklefile = '/...._10000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/yourVIDEO.mp4'
#if you want occlusions filled in, tell us how many frames to fill in, i.e. if there is a gap in data:
framestofill = 0. #note, put "0" if you want ALL gaps filled!

%matplotlib inline

from deeplabcut import refine_tracklets
TrackletManager, TrackletVisualizer = refine_tracklets(path_config_file, 
                                                          picklefile, 
                                                          Specific_videofile, 
                                                          min_swap_frac=0,
                                                          min_tracklet_frac=0, 
                                                          max_gap=framestofill)
'''

## Let's assume you have great tracking parameters, and you want to analyze a full set of videos:

In [ ]:
#^^^^^^^^^You do NOT neeed to run if you hit "save" in the GUI ^^^^^^^^^^
#this is just if you want to run the same parameters over a set of videos!

# You need to point to your pickle file, please "copy path" from the folder to the left (right click, copy path)
picklefile = '/content/drive/My Drive/mwm-penguins-2020-03-31/videos/penguindemoDLC_resnet50_mwmMar31shuffle1_22000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/content/drive/My Drive/mwm-penguins-2020-03-31/videos/penguindemo.mp4'

deeplabcut.convert_raw_tracks_to_h5(path_config_file, picklefile)
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType, 
                                 track_method = tracktype)

## Create plots of your trajectories:

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, track_method=tracktype)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                draw_skeleton=True, 
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True, 
                                track_method = tracktype)

## (OLD) Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
#create a path variable that links to the config file:
videofile_path = ['/content/Pilot5/Pilot05-2_CUT.mp4']
print(videofile_path)

In [ ]:
deeplabcut.analyze_videos(path_config_file, videofile_path, save_as_csv=True)

## (OLD) Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path)
#, videotype=VideoType

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## (OLD) Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)
#, videotype=VideoType